In [13]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
from functools import reduce
import operator
import json
import sys
sys.path.append('../')
import src.functions as func

### Choosing 3 cities to place my office:

#### A. Madrid:

In [14]:
madrid=func.geocode("Castellana 89, Paseo de la Castellana, Madrid") #Torre Picasso.

In [15]:
madrid

{'type': 'Point', 'coordinates': ['40.44980', '-3.69164']}

#### B. Málaga:

#### C. Barcelona:

### Prioritizing some requirements over othes:

- **Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.**
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- **Executives like Starbucks A LOT. Ensure there's a starbucks not too far.**
- **Account managers need to travel a lot.**
- **Everyone in the company is between 25 and 40, give them some place to go party.**
- **The CEO is vegan.**
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

### API Foursquare requests:

In [19]:
load_dotenv()

True

In [20]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [21]:
url = "https://favqs.com/api/session"
url_query = 'https://api.foursquare.com/v2/venues/search'

#### A. Madrid-Empresas de diseño:

In [22]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "empresas de diseño"
}

In [23]:
dis_mad = requests.get(url_query, params = parametros).json()

In [24]:
dis_mad

{'meta': {'code': 200, 'requestId': '618fc3103f7f6276a88413e9'},
 'response': {'venues': [{'id': '4c933ba32bf7b60c5f37690a',
    'name': 'Esne (Escuela Universitaria de Diseño, Innovación y Tecnología)',
    'contact': {},
    'location': {'address': 'Av de Alfonso XIII, 97',
     'lat': 40.45709578223368,
     'lng': -3.6673646435875544,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.45709578223368,
       'lng': -3.6673646435875544}],
     'distance': 2210,
     'postalCode': '28016',
     'cc': 'ES',
     'neighborhood': 'Chamartín',
     'city': 'Madrid',
     'state': 'Madrid',
     'country': 'España',
     'formattedAddress': ['Av de Alfonso XIII, 97',
      '28016 Madrid Madrid',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d1ae941735',
      'name': 'University',
      'pluralName': 'Universities',
      'shortName': 'University',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
       'suffix': '.png'},
      '

In [25]:
diseños_md = dis_mad["response"]["venues"]

In [67]:
#diseños_md

In [27]:
lista_diseñomad=func.extraetodo(diseños_md)

In [ ]:
datos_diseñomadrid = pd.DataFrame(lista_diseñomad)


In [ ]:
datos_diseñomadrid.head()

In [ ]:
diseñoMadrid = datos_diseñomadrid.copy()

#### A. Madrid-Starbucks:

In [30]:
parametros_St = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Starbucks"
}

In [31]:
St_mad = requests.get(url_query, params = parametros_St).json()

In [32]:
#St_mad

In [33]:
St_datamd = St_mad["response"]["venues"]

In [34]:
lista_starbucksmad=func.extraetodo(St_datamd)

In [35]:
datos_Starbucksmadrid=pd.DataFrame(lista_starbucksmad)

In [36]:
datos_Starbucksmadrid.head()

,nombre,latitud,longitud,location
0,Starbucks,40.447489,-3.693093,"{'type': 'Point', 'coordinates': [40.447488723..."
1,Starbucks,40.447729,-3.695175,"{'type': 'Point', 'coordinates': [40.447728697..."
2,Starbucks,40.455046,-3.693885,"{'type': 'Point', 'coordinates': [40.455046, -..."
3,Starbucks,40.432968,-3.683220,"{'type': 'Point', 'coordinates': [40.432967606..."
4,Starbucks,40.433564,-3.686829,"{'type': 'Point', 'coordinates': [40.433563917..."


In [37]:
StarbucksMadrid = datos_Starbucksmadrid.copy()

#### A. Madrid-Airports:

In [38]:
parametros_A = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Airport"
}

In [39]:
airport_mad=requests.get(url_query, params=parametros_A).json()

In [40]:
#airport_mad

In [41]:
air_datamd = airport_mad["response"]["venues"]

In [42]:
lista_airportmad=func.extraetodo(air_datamd)

In [43]:
datos_airportmadrid=pd.DataFrame(lista_airportmad)

In [44]:
datos_airportmadrid.head()

,nombre,latitud,longitud,location
0,AIRPORT TAVERN,40.454018,-3.678821,"{'type': 'Point', 'coordinates': [40.454017958..."
1,Hotel NH Madrid Barajas Airport,40.451022,-3.594986,"{'type': 'Point', 'coordinates': [40.451022341..."
2,Tryp Madrid Airport Suites,40.445569,-3.543607,"{'type': 'Point', 'coordinates': [40.4455692, ..."
3,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
4,Saludando – Hilton Madrid Airport,40.452070,-3.585533,"{'type': 'Point', 'coordinates': [40.452070242..."


#### A. Madrid-Disco:

In [45]:
parametros_D = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Disco"
}

In [46]:
disco_mad=requests.get(url_query,params=parametros_D).json()

In [47]:
disco_datamad=disco_mad["response"]["venues"]

In [48]:
lista_discomad=func.extraetodo(disco_datamad)

In [49]:
datos_discomadrid=pd.DataFrame(lista_discomad)

In [50]:
datos_discomadrid.head()

,nombre,latitud,longitud,location
0,Disco Rug & Play,40.451888,-3.696266,"{'type': 'Point', 'coordinates': [40.451888, -..."
1,Panamericana Disco,40.452806,-3.690264,"{'type': 'Point', 'coordinates': [40.452806048..."
2,Disco Express,40.434539,-3.710257,"{'type': 'Point', 'coordinates': [40.434538914..."
3,Disco Yedra Pub,40.434041,-3.714618,"{'type': 'Point', 'coordinates': [40.434040912..."
4,Disco Chiki,40.436151,-3.662824,"{'type': 'Point', 'coordinates': [40.436151496..."


####  A. Madrid-Vegan:

In [51]:
parametros_V= {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Vegan"
}

In [52]:
vegan_mad=requests.get(url_query,params=parametros_V).json()

In [53]:
vegan_datamad=vegan_mad["response"]["venues"]

In [54]:
lista_veganmad=func.extraetodo(vegan_datamad)

In [55]:
datos_veganmadrid=pd.DataFrame(lista_veganmad)

In [56]:
datos_veganmadrid.head()

,nombre,latitud,longitud,location
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,"{'type': 'Point', 'coordinates': [40.4424564, ..."
1,Vegan Streets Madrid Centro,40.417512,-3.697636,"{'type': 'Point', 'coordinates': [40.4175117, ..."
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -..."
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -..."
4,Los Andenes Vegan,40.435206,-3.710348,"{'type': 'Point', 'coordinates': [40.435206, -..."


#### Suming up all requirements in a single list (Madrid).

In [58]:
MADRID=func.sum_requirements(lista_diseñomad, lista_starbucksmad, lista_airportmad, lista_discomad, lista_veganmad)

In [60]:
datos_MADRID=pd.DataFrame(MADRID)

In [63]:
datos_MADRID.sample()

,nombre,latitud,longitud,location
115,El Final Disco,40.426136,-3.655518,"{'type': 'Point', 'coordinates': [40.426136, -..."


### Exporting data to Mongo DB Compass.

In [65]:
with open ("Madrid.json","w") as f: 
    json.dump(MADRID,f) 